## Simulation Verification

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import logging
import os
from collections import OrderedDict

import h5py
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from agnfinder import simulation_samples, simulation_utils
from agnfinder.lfi import main

In [3]:
os.chdir('/home/mike/repos/agnfinder')
save_loc = 'data/photometry_simulation_1000_verification.hdf5'

In [4]:
simulation_samples.simulate(n_samples=1000, save_loc=save_loc, emulate_ssp=True, noise=False)

log_mass 8.003262676608296 11.999643679261325
dust2 0.007270496860767702 13.79314432580479
tage 0.00010610024557038634 1.998868300449978
tau 0.11944335770649472 29.987243081951668
log_agn_mass -6.99792610448215 1.1716858109527326
agn_eb_v 0.0003521469542768161 0.4996893272183885
log_agn_torus_mass -6.994184631008644 1.170650763946952


[<class 'sedpy.observate.Filter'>(u_sloan), <class 'sedpy.observate.Filter'>(g_sloan), <class 'sedpy.observate.Filter'>(r_sloan), <class 'sedpy.observate.Filter'>(i_sloan), <class 'sedpy.observate.Filter'>(z_sloan), <class 'sedpy.observate.Filter'>(VISTA_H), <class 'sedpy.observate.Filter'>(VISTA_J), <class 'sedpy.observate.Filter'>(VISTA_Ks), <class 'sedpy.observate.Filter'>(VISTA_Y), <class 'sedpy.observate.Filter'>(VISTA_Z), <class 'sedpy.observate.Filter'>(wise_w1), <class 'sedpy.observate.Filter'>(wise_w2)]


 /media/mike/Windows/linux_cache/py36/lib/python3.6/site-packages/sklearn/base.py:253: UserWarning:Trying to unpickle estimator PCA from version 0.21.2 when using version 0.20.3. This might lead to breaking code or invalid results. Use at your own risk.
1000it [00:52, 19.01it/s]


In [5]:
assert False

AssertionError: 

In [ ]:
data_loc = save_loc
logging.warning('Using data loc {}'.format(data_loc))
assert os.path.isfile(data_loc)
with h5py.File(data_loc, 'r') as f:
    norm_theta = f['samples']['normalised_theta'][...]
    theta = f['samples']['theta'][...]
    # hacky extra normalisation here, not great TODO
#     simulated_y = -1 * np.log10(f['samples']['simulated_y'][...])
    simulated_y = f['samples']['simulated_y'][...]

In [ ]:
galaxy_index = 10

In [ ]:
theta[galaxy_index], norm_theta[galaxy_index], simulated_y[galaxy_index]

In [ ]:
direct_simulator, wavelengths = simulation_samples.get_photometry_simulator(
        emulate_ssp=True,
        noise=True)

In [ ]:
direct_photometry = direct_simulator(theta[galaxy_index])

In [ ]:
wavelengths

In [ ]:
sims = np.array([simulated_y[n] for n in range(100)])
assert np.argmin(np.sum(np.abs(sims - direct_photometry), axis=1)) == galaxy_index

In [ ]:
plt.loglog(wavelengths, sims[galaxy_index], 'x', label='stored')
plt.loglog(wavelengths, direct_photometry, 'x', label='recovered')
plt.legend() # also matches indirect (i.e. via normalisasimulate(args.n_samples, save_loc, args.emulate_ssp, args.noise)tion)

In [ ]:
assert False

In [ ]:
limits = OrderedDict({
            'log_mass': [8, 12], 
            'dust2': [0.001, 13.8],
            'tage': [0., 2.],
            'tau': [.1, 30],
            'log_agn_mass': [-7, np.log10(15)],
            'agn_eb_v': [0., 0.5],
            'log_agn_torus_mass': [-7, np.log10(15)]
        })

simulation_utils.denormalise_theta(norm_theta[0], limits)

In [ ]:
problem = main.load_simulator(emulate_ssp=True, noise=False)

In [ ]:
10 ** problem.simulator(norm_theta[0])